In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[10],8
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, -1, 0]"
ng_refs,-,"[3, 2, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.118864                    1 -0.139839
0.000750        2 -0.117997                    2 -0.139044
0.001052        3 -0.115269                    3 -0.137077
0.001476        4 -0.112568                    4 -0.135123
0.002070        5 -0.110018                    5 -0.133113
0.002904        6 -0.107682                    6 -0.131230
0.004074        7 -0.105689                    7 -0.129334
0.005714        8 -0.104155                    8 -0.127612
0.008015        9 -0.103292                    9 -0.126178
0.011243       10 -0.103328                   10 -0.125034
0.015771       11 -0.104607                   11 -0.124505
0.022122       12 -0.107622                   12 -0.124679
0.031031       13 -0.113104                   13 -0.126143
0.043528       14 -0.121142                   14 -0.128385
0.061057       15 -0.119447                   15 -0.122666
0.085645       16 -0.097580                   16 -0.100967
0.120136       17 -0.063157                   17 -0.067090
0.168516       18 -0.022470                   18 -0.025097
0.236378       19  0.025437                   19  0.025621
0.331549       20  0.083077                   20  0.086938
0.465100       21  0.157618                   21  0.163292
0.652400       22  0.248863                   22  0.248057
0.915100       23  0.252229                   23  0.248935
1.283650       24  0.178745                   24  0.173347
1.800600       25  0.117896                   25  0.112040
2.525700       26  0.070128                   26  0.069268
3.542800       27  0.044457                   27  0.047683
4.969550       28  0.031931                   28  0.034741
6.970850       29  0.024893                   29  0.026170
9.778100       30  0.020104                   30  0.020771
13.715850      31  0.016195                   31  0.017701
19.239350      32  0.012686                   32  0.014767
26.987250      33  0.009528                   33  0.011482
37.855300      34  0.006816                   34  0.008553
53.100050      35  0.001700                   35  0.002878
73.887500      36 -0.004887                   36 -0.005240
97.662500      37 -0.009644                   37 -0.011040
121.437500     38 -0.010153                   38 -0.011287
145.212500     39 -0.006662                   39 -0.007074
168.987500     40 -0.002946                   40 -0.003063
192.762500     41  0.002896                   41  0.002916
216.537500     42  0.016067                   42  0.019483
240.312500     43  0.035440                   43  0.047959
264.087500     44  0.060125                   44  0.062797
287.862500     45  0.088458                   45  0.063112
311.637500     46  0.117278                   46  0.089929
335.412500     47  0.142320                   47  0.134383
359.187500     48  0.157582                   48  0.162596
382.962500     49  0.157965                   49  0.158585
406.737500     50  0.157831                   50  0.154137
430.512500     51  0.154624                   51  0.157770
454.287500     52  0.148156                   52  0.161410
478.062500     53  0.138975                   53  0.162848
501.837500     54  0.127845                   54  0.147663
525.612500     55  0.115959                   55  0.131170
549.387500     56  0.103661                   56  0.104646
573.162500     57  0.091681                   57  0.086187
596.937500     58  0.080798                   58  0.070709
620.712500     59  0.072027                   59  0.062384
644.487500     60  0.068042                   60  0.060999
668.262500     61  0.071532                   61  0.066962
692.037500     62  0.079755                   62  0.077517
715.812500     63  0.078998                   63  0.073755
739.587500     64  0.059676                   64  0.056735
763.362500     65  0.026843                   65  0.022252
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug       fldg     fnetg       flug          fldg   
pressure    level                                                            
0.000000    1      -7.715035   0.000000 -7.715035  -7.574402 -1.178256e-08   
0.000624    2      -7.715048   0.000004 -7.715044  -7.574415  2.822928e-06   
0.000876    3      -7.715053   0.000006 -7.715047  -7.574421  3.980243e-06   
0.001229    4      -7.715060   0.000009 -7.715052  -7.574428  5.651840e-06   
0.001723    5      -7.715071   0.000012 -7.715059  -7.574438  8.065942e-06   
0.002417    6      -7.715085   0.000017 -7.715068  -7.574453  1.155299e-05   
0.003391    7      -7.715105   0.000025 -7.715080  -7.574473  1.658244e-05   
0.004757    8      -7.715132   0.000035 -7.715097  -7.574501  2.383948e-05   
0.006672    9      -7.715171   0.000050 -7.715121  -7.574541  3.429618e-05   
0.009359    10     -7.715225   0.000071 -7.715154  -7.574596  4.933464e-05   
0.013128    11     -7.715301   0.000101 -7.715200  -7.574673  7.093560e-05   
0.018415    12     -7.715408   0.000142 -7.715265  -7.574782  1.018546e-04   
0.025830    13     -7.715560   0.000200 -7.715360  -7.574936  1.460011e-04   
0.036232    14     -7.715777   0.000278 -7.715499  -7.575154  2.086562e-04   
0.050823    15     -7.716093   0.000385 -7.715709  -7.575465  2.978477e-04   
0.071291    16     -7.716545   0.000547 -7.715998  -7.575902  4.370075e-04   
0.100000    17     -7.717160   0.000830 -7.716330  -7.576491  6.828563e-04   
0.140271    18     -7.717967   0.001336 -7.716631  -7.577258  1.129325e-03   
0.196760    19     -7.718988   0.002207 -7.716782  -7.578218  1.921361e-03   
0.275997    20     -7.720208   0.003665 -7.716543  -7.579343  3.287830e-03   
0.387100    21     -7.721494   0.006045 -7.715449  -7.580486  5.574618e-03   
0.543100    22     -7.722380   0.009843 -7.712537  -7.581174  9.280883e-03   
0.761700    23     -7.721634   0.015542 -7.706092  -7.580334  1.486462e-02   
1.068500    24     -7.718865   0.021939 -7.696926  -7.577623  2.120128e-02   
1.498800    25     -7.714936   0.027121 -7.687815  -7.573799  2.621393e-02   
2.102400    26     -7.710141   0.030756 -7.679385  -7.569122  2.954885e-02   
2.949000    27     -7.705111   0.032759 -7.672352  -7.564197  3.157134e-02   
4.136600    28     -7.700015   0.033917 -7.666098  -7.559170  3.325251e-02   
5.802500    29     -7.694784   0.034987 -7.659797  -7.553978  3.491742e-02   
8.139200    30     -7.689338   0.036432 -7.652906  -7.548480  3.666358e-02   
11.417000   31     -7.683626   0.038525 -7.645100  -7.542584  3.883320e-02   
16.014700   32     -7.677654   0.041374 -7.636280  -7.536181  4.207211e-02   
22.464000   33     -7.671554   0.044965 -7.626588  -7.529340  4.651453e-02   
31.510500   34     -7.665589   0.049210 -7.616378  -7.522260  5.174047e-02   
44.200100   35     -7.659923   0.053791 -7.606133  -7.515074  5.741217e-02   
62.000000   36     -7.658706   0.056159 -7.602547  -7.512245  6.065165e-02   
85.775000   37     -7.671281   0.054969 -7.616312  -7.524408  5.805562e-02   
109.550000  38     -7.698366   0.054895 -7.643471  -7.553164  5.571462e-02   
133.325000  39     -7.735633   0.063566 -7.672066  -7.593413  6.417351e-02   
157.100000  40     -7.776711   0.085883 -7.690828  -7.637731  8.856593e-02   
180.875000  41     -7.822204   0.123079 -7.699125  -7.686660  1.288662e-01   
204.650000  42     -7.881532   0.190563 -7.690969  -7.750326  2.007446e-01   
228.425000  43     -7.986523   0.340805 -7.645718  -7.860867  3.661612e-01   
252.200000  44     -8.151701   0.605794 -7.545907  -8.032652  6.730285e-01   
275.975000  45     -8.389547   1.012971 -7.376576  -8.279451  1.096702e+00   
299.750000  46     -8.709339   1.581890 -7.127449  -8.609330  1.604341e+00   
323.525000  47     -9.116981   2.319824 -6.797157  -9.022447  2.270753e+00   
347.300000  48     -9.616240   3.219900 -6.396340  -9.524489  3.151299e+00   
371.075000  49    -10.206401   4.253861 -5.9525

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')